<a href="https://colab.research.google.com/github/kridtapon/CycleEdge-Oscillator-Strategy/blob/main/CycleEdge_Oscillator_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.2 MB/s eta 0:00:00


In [ ]:
pip install --upgrade yfinance

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# ---------------------------- #
# 1. Helper function: Stochastic RSI
# ---------------------------- #
def calculate_stochrsi(close, rsi_period=14, stoch_period=14):
    rsi = close.rolling(rsi_period).apply(lambda x: pd.Series(x).pct_change().add(1).apply(np.log).sum(), raw=False)
    min_rsi = rsi.rolling(stoch_period).min()
    max_rsi = rsi.rolling(stoch_period).max()
    stochrsi = (rsi - min_rsi) / (max_rsi - min_rsi)
    return stochrsi, rsi

# ---------------------------- #
# 2. Download Data
# ---------------------------- #
symbol = 'META'
start_date = '2019-01-01'
end_date = '2025-01-01'

df = yf.download(symbol, start=start_date, end=end_date, multi_level_index=False)

# ---------------------------- #
# 3. Calculate StochRSI + RSI
# ---------------------------- #
df['StochRSI'], df['RSI'] = calculate_stochrsi(df['Close'])

# ---------------------------- #
# 4. Generate Entry & Exit Signals
# ---------------------------- #
# Long Entry Signal
df['Entry_1'] = (
    (df['StochRSI'].shift(1) < 0.2) &
    (df['StochRSI'] >= 0.2) &
    (df['RSI'] < 30)
)

# Short Exit
df['Entry_2'] = (
    (df['StochRSI'] < 0.2) |
    ((df['StochRSI'].shift(1) > 0.5) & (df['StochRSI'] <= 0.5))
)

# Short Entry Signal
df['Exit_1'] = (
    (df['StochRSI'].shift(1) > 0.8) &
    (df['StochRSI'] <= 0.8) &
    (df['RSI'] > 70)
)

# Long Exit
df['Exit_2'] = (
    (df['StochRSI'] > 0.8) |
    ((df['StochRSI'].shift(1) < 0.5) & (df['StochRSI'] >= 0.5))
)

# ---------------------------- #
# 5. Combine Entry & Exit Logic
# ---------------------------- #
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]
df['Entry'] = df['Entry_1'] | df['Entry_2']
df['Exit'] = df['Exit_1'] | df['Exit_2']

# ---------------------------- #
# 6. Filter Period & Build Portfolio
# ---------------------------- #

entries = df['Entry'].shift(1).fillna(False).astype(bool).to_numpy()
exits = df['Exit'].shift(1).fillna(False).astype(bool).to_numpy()

portfolio = vbt.Portfolio.from_signals(
    close=df['Open'],
    entries=entries,
    exits=exits,
    init_cash=100_000,
    fees=0.001,
    slippage=0.002,
    freq='D'
)

# ---------------------------- #
# 7. Results
# ---------------------------- #
print(portfolio.stats())
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
<ipython-input-8-d11a7b0ce8ad>:63: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-8-d11a7b0ce8ad>:64: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-8-d11a7b0ce8ad>:70: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_do

Start                               2020-01-02 00:00:00
End                                 2024-12-31 00:00:00
Period                               1258 days 00:00:00
Start Value                                    100000.0
End Value                                 345729.718598
Total Return [%]                             245.729719
Benchmark Return [%]                         187.561082
Max Gross Exposure [%]                            100.0
Total Fees Paid                            31933.544154
Max Drawdown [%]                              45.401806
Max Drawdown Duration                 407 days 00:00:00
Total Trades                                         90
Total Closed Trades                                  89
Total Open Trades                                     1
Open Trade PnL                            -11646.920925
Win Rate [%]                                  64.044944
Best Trade [%]                                23.289567
Worst Trade [%]                              -21